In [218]:
from bokeh.io import output_file, show
from bokeh.models import ColorBar, ColumnDataSource, HoverTool, LinearColorMapper
from bokeh.palettes import RdBu
from bokeh.plotting import figure
from bokeh.sampledata.periodic_table import elements
from bokeh.transform import dodge, factor_cmap, transform

import numpy as np
import pandas as pd

import temann
from temann.data import TEMDB

In [219]:
TEMDB.columns

Index(['ID', 'T(K)', 'Z*10^-4 reported', 'Resist(Ohm.cm)', 'Seebeck(uV/K)',
       'kappaZT', 'Unnamed: 6', 'Resist(400K)', 'Seebeck(400K)', 'Pf(W/K^2/m)',
       'ZT', 'kappa(W/mK)', 'x', 'Formula', 'series', 'T_Max', 'family',
       'Unnamed: 17', 'Unnamed: 18', 'Conduct(S/cm)', 'Power_Factor*T_(W/mK)',
       'preparative_route', 'final_form', 'cell_volume(A^3)',
       'formula_units_per_cell', 'atoms_per_formula_unit',
       'total_atoms_per_unit_cell', 'average_atomic_volume',
       'ICSD_of_structure', 'temp of ICSD (K)', 'S^2', 'ke/ktotal',
       'space_group', 'number_symmetry_elements', 'Unnamed: 34',
       'Unnamed: 35'],
      dtype='object')

In [220]:
data = TEMDB[['T(K)', 'Seebeck(uV/K)', 'Formula']]

In [221]:
data.head()

,T(K),Seebeck(uV/K),Formula
0,300.0,-462.967,CaMnO3
1,400.0,-462.967,CaMnO3
2,700.0,-470.651,CaMnO3
3,1000.0,-279.829,CaMnO3
4,300.0,-201.708,Ca0.98La0.02MnO3


In [222]:
data.columns = ['T', 'seebeck', 'formula']

In [223]:
data.dropna(inplace=True)

In [224]:
data.reset_index(drop=True, inplace=True)

In [225]:
data.head()

,T,seebeck,formula
0,300.0,-462.967,CaMnO3
1,400.0,-462.967,CaMnO3
2,700.0,-470.651,CaMnO3
3,1000.0,-279.829,CaMnO3
4,300.0,-201.708,Ca0.98La0.02MnO3


In [227]:
goodS = data[(data['seebeck'] > 150) | (data['seebeck'] < -150)]
goodS.reset_index(drop=True, inplace=True)

In [229]:
elems = []

for i in range(len(goodS)):
    split_formula = temann.get_empirical_formula(goodS.loc[i, 'formula'])
    elems.append([i, *list(split_formula.keys())])

In [230]:
eles = pd.DataFrame(elems, columns=['index', 'element1', 'element2', 'element3', 'element4', 'element5'])

In [231]:
eles.set_index('index', inplace=True)

In [232]:
data[['element1', 'element2', 'element3', 'element4', 'element5']] = eles

In [233]:
sums = data[['element1', 'element2', 'element3', 'element4', 'element5']].apply(pd.Series.value_counts).sum(axis=1)

In [234]:
sums = pd.DataFrame(sums, columns=['counts'], dtype=int)
sums.index.name = 'symbol'


In [235]:
elements = elements.join(sums, on='symbol')

In [236]:
elements['counts'].fillna(value=0, inplace=True)

In [237]:
elements.head()

,atomic number,symbol,name,atomic mass,CPK,electronic configuration,electronegativity,atomic radius,ion radius,van der Waals radius,...,standard state,bonding type,melting point,boiling point,density,metal,year discovered,group,period,counts
0,1,H,Hydrogen,1.00794,#FFFFFF,1s1,2.20,37.0,NaN,120.0,...,gas,diatomic,14.0,20.0,0.00009,nonmetal,1766,1,1,0.0
1,2,He,Helium,4.002602,#D9FFFF,1s2,NaN,32.0,NaN,140.0,...,gas,atomic,NaN,4.0,0.00000,noble gas,1868,18,1,0.0
2,3,Li,Lithium,6.941,#CC80FF,[He] 2s1,0.98,134.0,76 (+1),182.0,...,solid,metallic,454.0,1615.0,0.54000,alkali metal,1817,1,2,7.0
3,4,Be,Beryllium,9.012182,#C2FF00,[He] 2s2,1.57,90.0,45 (+2),NaN,...,solid,metallic,1560.0,2743.0,1.85000,alkaline earth metal,1798,2,2,0.0
4,5,B,Boron,10.811,#FFB5B5,[He] 2s2 2p1,2.04,82.0,27 (+3),NaN,...,solid,covalent network,2348.0,4273.0,2.46000,metalloid,1807,13,2,3.0


In [238]:
elements.loc[elements['symbol'] == 'O', 'counts'] = 0

In [239]:
output_file("luke_periodic.html")

periods = ["I", "II", "III", "IV", "V", "VI", "VII"]
groups = [str(x) for x in range(1, 19)]

df = elements.copy()
df["atomic mass"] = df["atomic mass"].astype(str)
df["group"] = df["group"].astype(str)
df["period"] = [periods[x-1] for x in df.period]
df = df[df.group != "-"]
df = df[df.symbol != "Lr"]
df = df[df.symbol != "Lu"]

# cmap = {
#     "alkali metal"         : "#a6cee3",
#     "alkaline earth metal" : "#1f78b4",
#     "metal"                : "#d93b43",
#     "halogen"              : "#999d9a",
#     "metalloid"            : "#e08d49",
#     "noble gas"            : "#eaeaea",
#     "nonmetal"             : "#f1d4Af",
#     "transition metal"     : "#599d7A",
# }

# colors = ["#75968f", "#a5bab7", "#c9d9d3", "#e2e2e2", "#dfccce", "#ddb7b1", "#cc7878", "#933b41", "#550b1d"]
colors = RdBu[11]
mapper = LinearColorMapper(palette=colors, low=df['counts'].min(), high=df['counts'].max())

source = ColumnDataSource(df)

p = figure(title="Periodic Table (omitting LA and AC Series)", plot_width=1000, plot_height=450,
           tools="", toolbar_location=None,
           x_range=groups, y_range=list(reversed(periods)))

p.rect("group", "period", 0.95, 0.95, source=source, fill_alpha=0.6, legend="metal",
       fill_color=transform('counts', mapper))
       #color=factor_cmap('metal', palette=list(cmap.values()), factors=list(cmap.keys())))

text_props = {"source": source, "text_align": "left", "text_baseline": "middle"}

x = dodge("group", -0.4, range=p.x_range)

r = p.text(x=x, y="period", text="symbol", **text_props)
r.glyph.text_font_style="bold"

r = p.text(x=x, y=dodge("period", 0.3, range=p.y_range), text="atomic number", **text_props)
r.glyph.text_font_size="8pt"

r = p.text(x=x, y=dodge("period", -0.35, range=p.y_range), text="name", **text_props)
r.glyph.text_font_size="5pt"

r = p.text(x=x, y=dodge("period", -0.2, range=p.y_range), text="atomic mass", **text_props)
r.glyph.text_font_size="5pt"

p.text(x=["3", "3"], y=["VI", "VII"], text=["LA", "AC"], text_align="center", text_baseline="middle")

p.add_tools(HoverTool(tooltips = [
    ("Name", "@name"),
    ("Atomic number", "@{atomic number}"),
    ("Atomic mass", "@{atomic mass}"),
    ("Type", "@metal"),
    ("CPK color", "$color[hex, swatch]:CPK"),
    ("Electronic configuration", "@{electronic configuration}"),
]))

p.outline_line_color = None
p.grid.grid_line_color = None
p.axis.axis_line_color = None
p.axis.major_tick_line_color = None
p.axis.major_label_standoff = 0
p.legend.orientation = "horizontal"
p.legend.location ="top_center"

show(p)

In [49]:
import pandas as pd

from bokeh.io import output_file, show
from bokeh.models import BasicTicker, ColorBar, ColumnDataSource, LinearColorMapper, PrintfTickFormatter
from bokeh.plotting import figure
from bokeh.sampledata.unemployment1948 import data
from bokeh.transform import transform

output_file("unemploymemt.html")

data.Year = data.Year.astype(str)
data = data.set_index('Year')
data.drop('Annual', axis=1, inplace=True)
data.columns.name = 'Month'

# reshape to 1D array or rates with a month and year for each row.
df = pd.DataFrame(data.stack(), columns=['rate']).reset_index()

source = ColumnDataSource(df)

# this is the colormap from the original NYTimes plot
colors = ["#75968f", "#a5bab7", "#c9d9d3", "#e2e2e2", "#dfccce", "#ddb7b1", "#cc7878", "#933b41", "#550b1d"]
mapper = LinearColorMapper(palette=colors, low=df.rate.min(), high=df.rate.max())

p = figure(plot_width=800, plot_height=300, title="US Unemployment 1948—2016",
           x_range=list(data.index), y_range=list(reversed(data.columns)),
           toolbar_location=None, tools="", x_axis_location="above")

p.rect(x="Year", y="Month", width=1, height=1, source=source,
       line_color=None, fill_color=transform('rate', mapper))

color_bar = ColorBar(color_mapper=mapper, location=(0, 0),
                     ticker=BasicTicker(desired_num_ticks=len(colors)),
                     formatter=PrintfTickFormatter(format="%d%%"))

p.add_layout(color_bar, 'right')

p.axis.axis_line_color = None
p.axis.major_tick_line_color = None
p.axis.major_label_text_font_size = "5pt"
p.axis.major_label_standoff = 0
p.xaxis.major_label_orientation = 1.0

show(p)

In [19]:
np.ndarray

numpy.ndarray

In [44]:
x = np.random.normal(size=40)

In [98]:
df = None
df = pd.DataFrame(x, columns=['x'])
df['y'] = np.ones(len(x))
# df = pd.DataFrame(df.stack(), columns=['x']).reset_index()

output_file('luke_test.html')

source = ColumnDataSource(df)

colors = ["#75968f", "#a5bab7", "#c9d9d3", "#e2e2e2", "#dfccce", "#ddb7b1", "#cc7878", "#933b41", "#550b1d"]
mapper = LinearColorMapper(palette=colors, low=df.x.min(), high=df.x.max())

p = figure(plot_width=700, plot_height=200, title="Numbers Generated from Normal Distribution",
           x_range=list(df.index.astype(str)), y_range=list("x"),
           toolbar_location=None, tools="", x_axis_location="below")

p.rect(x="index", y="y", width=0.5, height=2, source=source,
       line_color=None, fill_color=transform('x', mapper))

color_bar = ColorBar(color_mapper=mapper, location=(0, 0),
                     ticker=BasicTicker(desired_num_ticks=len(colors)))

p.add_layout(color_bar, 'right')

show(p)

E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: x [renderer: GlyphRenderer(id='8468c3f8-bcb3-4f85-850d-938a098989a9', ...)]
E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: y [renderer: GlyphRenderer(id='a015dfb0-4063-4787-837c-3f8f841bfa20', ...)]
E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: y [renderer: GlyphRenderer(id='b021e5f7-1114-441c-98eb-49f170b42b0e', ...)]


In [74]:
# np.ones(40)
df['y'] = np.ones(len(x))